In [135]:
import pandas as pd

df_weather = {}
df_weather_extended = {}

filtered_df = {}
filtered_df_extended = {}

df_weather['nominal'] = []
df_weather_extended['nominal'] = []
df_weather['nominal'] = pd.read_csv('content/evaluation/model_original/per_class_metrics.csv')
df_weather_extended['nominal'] = pd.read_csv('content/evaluation/model_extended/per_class_metrics.csv')

weathers = ['fog', 'morning', 'overcast', 'rain', 'sunset']
for weather in weathers:
    df_weather[weather] = []
    df_weather_extended[weather] = []
    df_weather[weather] = pd.read_csv(f'content/evaluation/model_original_weather/per_class_metrics_{weather}.csv')
    df_weather_extended[weather] = pd.read_csv(f'content/evaluation/model_extended_weather/per_class_metrics_{weather}.csv')


weathers.insert(0,'nominal')
for weather in weathers:
    filtered_df[weather] = []
    filtered_df_extended[weather] = []
    filtered_df[weather] = df_weather[weather][df_weather[weather]['Class'].isin([0, 6, 7, 8, 9,  10, 11, 12, 13, 255])]
    filtered_df_extended[weather] = df_weather_extended[weather][df_weather_extended[weather]['Class'].isin([0, 6, 7, 8, 9,  10, 11, 12, 13, 255])]

results_df = {}
results_df_extended = {}

for weather in weathers:
    results_df[weather] = filtered_df[weather][['Class', 'IoU']].loc[df_weather[weather]['IoU'] != 0]
    results_df_extended[weather] = filtered_df_extended[weather][['Class', 'IoU']].loc[df_weather_extended[weather]['IoU'] != 0]
    
    average_iou_nominal = results_df[weather]['IoU'].mean()
    average_iou_extended = results_df_extended[weather]['IoU'].mean()
    print(f'{weather} & {average_iou_nominal:.3f} & {average_iou_extended:.3f} \\')
    


nominal & 0.663 & 0.718 \
fog & 0.365 & 0.520 \
morning & 0.667 & 0.709 \
overcast & 0.669 & 0.707 \
rain & 0.385 & 0.650 \
sunset & 0.685 & 0.748 \


In [132]:
import pandas as pd
import numpy as np
import math

df_weather = {}
df_weather_extended = {}

filtered_df = {}
filtered_df_extended = {}

df_weather['nominal'] = []
df_weather_extended['nominal'] = []
df_weather['nominal'] = pd.read_csv('content/evaluation/model_original/per_class_metrics.csv')


maximum_iou={}
minimum_iou={}
average_iou={}
variance_iou={}

perturbations=[]


names={
    'gaussian_noise': 'A-I',
    'poisson_noise': 'A-II',
    'impulse_noise': 'A-III',
    'jpeg_filter': 'A-IV',
    'speckle_noise_filter': 'A-V',
    'defocus_blur': 'B-I',
    'motion_blur': 'B-II',
    'gaussian_blur': 'B-IV',
    'low_pass_filter': 'B-V',
    'frost_filter': 'C-I',
    'snow_filter': 'C-II',
    'fog_filter': 'C-III',
    'increase_brightness': 'C-IV',
    'contrast': 'C-V',
    'elastic': 'D-I',
    'pixelate': 'D-II',
    'sample_pairing_filter': 'D-III',
    'sharpen_filter': 'D-IV',
    'scale_image': 'E-II',
    'translate_image': 'E-III',
    'splatter_mapping': 'F-I',
    'dotted_lines_mapping': 'F-II',
    'zigzag_mapping': 'F-III',
    'canny_edges_mapping': 'F-IV',
    'cutout_filter': 'F-V',
    'false_color_filter': 'G-I',
    'phase_scrambling': 'G-II',
    'histogram_equalisation': 'G-III',
    'white_balance_filter': 'G-IV',
    'grayscale_filter': 'G-V',
    'saturation_filter': 'G-VI',
    'saturation_decrease_filter': 'G-VI',
    'posterize_filter': 'G-VII'
}
perturbations=list(names.keys())
perturbations.insert(0,"nominal")

for perturbation in perturbations:
    weathers=[]
    
    for i in range(0,5):
        weathers.append(f"b'0020_{perturbation}_{i}'")
    if perturbation=='nominal':
        weathers=['nominal']
    else:
        for weather in weathers:
            df_weather[weather] = []
            df_weather[weather] = pd.read_csv(f'content/evaluation/model_pert_perclass/per_class_metrics_({weather}).csv')
    for weather in weathers:
        filtered_df[weather] = []
        filtered_df[weather] = df_weather[weather][df_weather[weather]['Class'].isin([0, 6, 7, 8, 9,  10, 11, 12, 13, 255])]

    results_df = {}
    results_df_extended = {}
    iou_list=[]
    for weather in weathers:
        results_df[weather] = filtered_df[weather][['Class', 'IoU']].loc[df_weather[weather]['IoU'] != 0]
        
        average_iou_nominal = results_df[weather]['IoU'].mean()
        iou_list.append(average_iou_nominal)
    max_value = max(iou_list)
    min_value = min(iou_list)
    average_value = sum(iou_list) / len(iou_list)
    variance_value = math.sqrt(np.var(iou_list))
    if weathers[0]!="nominal":
        weather=weathers[0].split("b'0020_")[1].split("_0")[0]
    else:
        weather=weathers[0]
    maximum_iou[weather]=max_value
    minimum_iou[weather]=min_value
    average_iou[weather]=average_value
    variance_iou[weather]=variance_value

for key in average_iou.keys():
    if key=="nominal":
        print(f"{key} & {average_iou[key]:.2f} \\")
    else:
        print(f"{names[key]} & {average_iou[key]:.2f} & {variance_iou[key]:.2f} & {maximum_iou[key]:.2f} & {minimum_iou[key]:.2f} \\")



nominal & 0.66 \
A-I & 0.43 & 0.16 & 0.64 & 0.23 \
A-II & 0.50 & 0.10 & 0.58 & 0.32 \
A-III & 0.41 & 0.16 & 0.62 & 0.21 \
A-IV & 0.59 & 0.02 & 0.62 & 0.56 \
A-V & 0.58 & 0.09 & 0.66 & 0.41 \
B-I & 0.57 & 0.06 & 0.65 & 0.47 \
B-II & 0.63 & 0.03 & 0.66 & 0.58 \
B-IV & 0.58 & 0.08 & 0.66 & 0.46 \
B-V & 0.63 & 0.01 & 0.64 & 0.62 \
C-I & 0.62 & 0.04 & 0.65 & 0.54 \
C-II & 0.64 & 0.04 & 0.67 & 0.56 \
C-III & 0.56 & 0.14 & 0.67 & 0.29 \
C-IV & 0.66 & 0.00 & 0.66 & 0.65 \
C-V & 0.66 & 0.00 & 0.66 & 0.65 \
D-I & 0.63 & 0.01 & 0.64 & 0.61 \
D-II & 0.62 & 0.05 & 0.66 & 0.53 \
D-III & 0.41 & 0.16 & 0.64 & 0.20 \
D-IV & 0.66 & 0.01 & 0.67 & 0.64 \
E-II & 0.41 & 0.14 & 0.60 & 0.20 \
E-III & 0.39 & 0.16 & 0.66 & 0.19 \
F-I & 0.65 & 0.01 & 0.66 & 0.64 \
F-II & 0.66 & 0.01 & 0.66 & 0.65 \
F-III & 0.66 & 0.00 & 0.66 & 0.65 \
F-IV & 0.63 & 0.00 & 0.64 & 0.63 \
F-V & 0.64 & 0.02 & 0.66 & 0.60 \
G-I & 0.50 & 0.09 & 0.58 & 0.34 \
G-II & 0.50 & 0.16 & 0.66 & 0.23 \
G-III & 0.65 & 0.01 & 0.66 & 0.64 \
G-IV & 